
# TradeRunner Streaming Demo (Two Gotobi Entries)
Simulate live streaming from the USDJPY parquet into Backtrader using two `GotobiBT` legs with different entry/exit times and sizes. Trade sizes are in contracts and multiplied by the broker contract size (see `config/contracts.json`).



## Prerequisites
- `backtrader`
- `pandas`
- Repo root on `PYTHONPATH`
- Contract sizes config at `config/contracts.json` (defaults: USDJPY/EURUSD/GBPUSD = 100,000)


In [1]:

import sys
from pathlib import Path

REPO_ROOT = Path(r"C:\Users\justi\OneDrive\Trading")  # <-- adjust if your path differs
sys.path.insert(0, str(REPO_ROOT))

In [2]:

from backend import (
    RunnerConfig,
    StrategySpec,
    TradeRunnerBuilder,
    DataHandler,
    DataStreamer,
)
from strategies.gotobi_bt import GotobiBT
from pprint import pprint


In [3]:

# Load sample data
DATA_PATH = Path(r"C:\Users\justi\OneDrive\Trading\data\usdjpy_1min_2024-01-01_2025-10-01.parquet")
if not DATA_PATH.exists():
    raise FileNotFoundError(f"Sample parquet not found: {DATA_PATH}")

handler = DataHandler()
df = handler.load_parquet(DATA_PATH)
print(df.head())
print("Rows:", len(df))


                              open     high      low    close  volume
datetime                                                             
2024-01-01 00:32:00+00:00  141.020  141.020  141.020  141.020       1
2024-01-01 00:33:00+00:00  140.952  140.952  140.952  140.952       1
2024-01-01 06:45:00+00:00  140.962  141.025  140.962  141.025       5
2024-01-01 06:46:00+00:00  141.026  141.026  141.025  141.025       2
2024-01-01 17:04:00+00:00  141.399  141.400  140.995  141.110       6
Rows: 644043


In [14]:

# Set up streaming source (use a subset to keep the demo quick)
streamer = DataStreamer()
bar_queue = streamer.subscribe()
stream_df = df
print("Streaming bars:", len(stream_df))

Streaming bars: 644043


In [15]:

# Custom no-settlement days (add your dates here)
custom_nosettle = set()


In [16]:

# Build TradeRunner with two Gotobi legs (sizes are contract counts; contract_size pulled from config/contracts.json)
spec = StrategySpec(
    symbol="USDJPY",
    strategy=GotobiBT,  # base strategy ref
    strategies=[
        (
            GotobiBT,
            {
                "entry_time": "09:00:00",
                "exit_time": "09:55:00",
                "use_holidays": True,
                "notrade_days": custom_nosettle,
                "gotobi_days": (5, 10, 15, 20, 25, 30),
                "trade_size": 1,  # 1 contract -> multiplied by 100,000 lot if config present
            },
        ),
        (
            GotobiBT,
            {
                "entry_time": "10:00:00",
                "exit_time": "10:45:00",
                "use_holidays": True,
                "notrade_days": custom_nosettle,
                "gotobi_days": (5, 10, 15, 20, 25, 30),
                "trade_size": -1,  # short 1 contract
            },
        ),
    ],
    data=bar_queue,  # queue will be wrapped by StreamingOHLCVFeed inside TradeRunner
    name="gotobi-usdjpy-stream",
    cash=1000,
    commission=0.00001,
    leverage=500,
)

config = RunnerConfig(
    mode="live",
    base_currency="USD",
    invert_fx_to_base=True,
    preload=False,
    contracts_config_path="config/contracts.json",
    default_contract_size_fx=100000,
)

pool = TradeRunnerBuilder().build([spec], config=config)
runner = pool.runners[0]

# Start strategy thread then stream bars in this thread
thread = runner.start_async()
streamer.from_dataframe(stream_df, delay_sec=0.0)  # push bars then send sentinel None on close
thread.join()


[2024-01-05 09:01:00] ORDER SELL gotobi-usdjpy-stream size=-100000 px=145.113
[2024-01-05 09:56:00] ORDER Margin gotobi-usdjpy-stream
[2024-01-05 10:01:00] ORDER Margin gotobi-usdjpy-stream
[2024-01-05 10:46:00] ORDER Margin gotobi-usdjpy-stream
[2024-01-08 09:56:00] ORDER BUY gotobi-usdjpy-stream size=100000 px=144.206
[2024-01-08 09:56:00] TRADE gotobi-usdjpy-stream PnL gross=90700.00 net=90410.68
[2024-01-10 09:01:00] ORDER SELL gotobi-usdjpy-stream size=-100000 px=144.979
[2024-01-10 09:56:00] ORDER BUY gotobi-usdjpy-stream size=100000 px=144.871
[2024-01-10 09:56:00] TRADE gotobi-usdjpy-stream PnL gross=10800.00 net=10510.15
[2024-01-10 10:01:00] ORDER BUY gotobi-usdjpy-stream size=100000 px=144.93
[2024-01-10 10:46:00] ORDER SELL gotobi-usdjpy-stream size=-100000 px=144.885
[2024-01-10 10:46:00] TRADE gotobi-usdjpy-stream PnL gross=-4500.00 net=-4789.82
[2024-01-15 09:01:00] ORDER SELL gotobi-usdjpy-stream size=-100000 px=145.446
[2024-01-15 09:56:00] ORDER BUY gotobi-usdjpy-stre

In [17]:

# Inspect results
cerebro = runner.cerebro
broker_value = cerebro.broker.getvalue() if cerebro else None
print("Final broker value:", broker_value)

# Access strategy/analyzer objects if needed
pprint(runner.results)


Final broker value: 106036.1300000024
